In [1]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys


from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


In [2]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [3]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [4]:
print(len(behavior_labels))
print(len(behavior_index_in_df))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
15651


Now we make our own SVM model

In [5]:
# SVM
clf_behavior_SVM = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
clf_behavior_SVM.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0))]

In [6]:
clf_behavior_SVM.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

Predict in testing set

In [7]:
# predict in testing set.
y_behavior_SVM = clf_behavior_SVM.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_SVM, y_test)
f1_score_SVM = computeF1Score(y_test, y_behavior_SVM, average = None)
print('F1 Score: ', f1_score_SVM)
auc_score_SVM = computeAUCBehavior(clf_behavior_SVM, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('SVM classifier AUC: ', auc_score_SVM)

length of total comps:  1272
587 different item in whole dataset
accuracy: 53.852201%
F1 Score:  [0.67790019 0.38523274 0.22222222]
SVM classifier AUC:  {'current': 0.7125589922086931, 'looking': 0.6873433680530938, 'reflecting': 0.6770377303033585, 'micro': 0.7393821570547051, 'macro': 0.6929394220980538}


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Test in training set to figure out why the test accuracy is low

In [8]:
# test in training set to figure out why the test accuracy is low
y_train_behavior_SVM = clf_behavior_SVM.predict(X_train)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_train_behavior_SVM, y_train)
f1_train_score_SVM = computeF1Score(y_train, y_train_behavior_SVM, average = None)
print('F1 Score: ', f1_train_score_SVM)
auc_score_train_SVM = computeAUCBehavior(clf_behavior_SVM, X_train, y_train, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('SVM classifier AUC: ', auc_score_train_SVM)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  5085
2131 different item in whole dataset
accuracy: 58.092429%
F1 Score:  [0.70460088 0.45981773 0.28909657]
SVM classifier AUC:  {'current': 0.761881645454171, 'looking': 0.7449805369604868, 'reflecting': 0.737953883441176, 'micro': 0.7789965957290468, 'macro': 0.7484219415178953}


We can see the performance in training set is also not good. So there must be sth wrong within the model structure parameters, since the model structure are same. Please see compareStructure.ipynb for visualization.

Update structure parameters in SVM model. To make sure they become same with the initial model

In [9]:
clf_behavior_SVM_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
params_SVM = {'features__text__tfidf__analyzer':'word', 'features__text__tfidf__dtype':np.int64, 
             'features__text__tfidf__max_features':94963, 'features__text__tfidf__ngram_range':(1,3) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_behavior_SVM_updateParams.set_params(**params_SVM)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [10]:
clf_behavior_SVM_updateParams.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

Predict in testing set

In [11]:
# predict in testing set.
y_behavior_SVM_updateParams = clf_behavior_SVM_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_SVM_updateParams, y_test)
f1_score_SVM_updateParams = computeF1Score(y_test, y_behavior_SVM_updateParams, average = None)
print('F1 Score: ', f1_score_SVM_updateParams)
auc_score_SVM_updateParams = computeAUCBehavior(clf_behavior_SVM_updateParams, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('SVM classifier AUC: ', auc_score_SVM_updateParams)

length of total comps:  1272
501 different item in whole dataset
accuracy: 60.613208%
F1 Score:  [0.69674556 0.55873926 0.42510121]
SVM classifier AUC:  {'current': 0.7556279496104347, 'looking': 0.8059197242969479, 'reflecting': 0.7646148848289839, 'micro': 0.7983400948340651, 'macro': 0.7759697748617878}


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
